In [1]:
import cv2
import numpy as np
from scipy.spatial import distance

In [2]:
net=cv2.dnn.readNet( "yolov3.weights","yolov3.cfg",)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
classes=[]
with open("coco.names","r") as f:
    classes=[line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0]-1]for i in net.getUnconnectedOutLayers()]

In [3]:
mouse_pts = []
def get_mouse_points(event, x, y, flags, param):
    global mouseX, mouseY, mouse_pts
    if event == cv2.EVENT_LBUTTONDOWN:
        mouseX, mouseY = x, y
        cv2.circle(image, (x, y), 5, (0, 128, 255), 5)
        if "mouse_pts" not in globals():
            mouse_pts = []
        mouse_pts.append((x, y))
        print("Point detected")
        print(mouse_pts)

In [4]:
def cam_perspective(img, src_points):
    IMAGE_H = img.shape[0]
    IMAGE_W = img.shape[1]
    src = np.float32(np.array(src_points))
    dst = np.float32([[0, 0],[IMAGE_W, 0],[0, IMAGE_H], [IMAGE_W, IMAGE_H]])
    M = cv2.getPerspectiveTransform(src, dst)
    return M

In [5]:
def bdi(frame):
    frame_h = frame.shape[0]
    frame_w = frame.shape[1]
    scale_w = 1.2 / 2
    scale_h = 3 / 2
    blank_image = np.zeros((int(frame_h * scale_h), int(frame_w * scale_w), 3), np.uint8)
    blank_image[:] = (0,0,0)
    warped_pts = []
    return blank_image, warped_pts

In [6]:
def bev_pts(blank_image, warped_pts,mid_point_x,mid_point_y, M, scale_w, scale_h):
    node_radius = 4
    color_node = (255, 255, 255)
    thickness_node = 6
    scale_w = 1.2 / 2
    scale_h = 3 / 2
    pts = np.array([[[mid_point_x, mid_point_y]]], dtype="float32")
    warped_pt = cv2.perspectiveTransform(pts, M)[0][0]
    warped_pt_scaled = [int(warped_pt[0] * scale_w), int(warped_pt[1] * scale_h)]
    warped_pts.append(warped_pt_scaled)
    bird_image = cv2.circle(blank_image,(warped_pt_scaled[0], warped_pt_scaled[1]),node_radius,color_node,thickness_node)
    return warped_pts, bird_image

In [7]:
cap=cv2.VideoCapture("TownCentrr.mp4")
scale_w = 1.2 / 2
scale_h = 3 / 2
four_points=[]
frame_num = 0
cv2.namedWindow("image")
cv2.setMouseCallback("image", get_mouse_points)
num_mouse_points = 0
first_frame_display = True
while True:
    frame_num += 1
    _,frame= cap.read()
    frame_h = frame.shape[0]
    frame_w = frame.shape[1]
    if frame_num == 1:
        while True:
            image = frame
            cv2.imshow("image", image)
            cv2.waitKey(1)
            if len(mouse_pts) == 7:
                cv2.destroyWindow("image")
                break
            first_frame_display = False
        four_points = mouse_pts
        M = cam_perspective(frame, four_points[0:4])
        pedestrian_detect = frame
        pts = np.array([four_points[0], four_points[1], four_points[3], four_points[2]], np.int32)
        sixft=distance.euclidean(four_points[4], four_points[5])
    cv2.polylines(frame, [pts], True, (0, 255, 255), thickness=4)
    blob=cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False)
    net.setInput(blob)
    outs= net.forward(outputlayers)
    #class_ids=[]
    confidences=[]
    boxes= []
    location=[]
    blank_image, warped_pts=bdi(frame)
    for out in outs:
        for detection in out:
            scores= detection[5:]
            class_id =np.argmax(scores)
            confidence=scores[class_id]
            if confidence > 0.1 and class_id == 0:
                center_x=int(detection[0]*frame_w)
                center_y=int(detection[1]*frame_h)
                w=int(detection[2]*frame_w)
                h=int(detection[3]*frame_h)
                x=int(center_x -w/2)
                y= int(center_y - h/2)
                
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                #class_ids.append(class_id)
    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)
    
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h= boxes[i]
            x_c=int(x+w/2)
            y_c=int(y+h/2)
            #label=str(classes[class_ids[i]])
            #confidence=confidences[i]
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            #cv2.putText(frame,str(round(confidence,2)),(x,y+30), cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
            warped_pts, bird_image = bev_pts(blank_image, warped_pts,x_c,y_c, M, scale_w, scale_h)
            location.append([x,y,w,h])
            #print(warped_pts)
            
    for i in range(len(location)):
        j=i+1
        #x,y,w,h= location[i]
        #cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        while j<len(warped_pts):
            diff= distance.euclidean(warped_pts[i], warped_pts[j])
            x1,y1,w1,h1= location[i]
            x2,y2,w2,h2= location[j]
            if diff<=sixft:
                x_c1=int(x1+w1/2)
                y_c1=int(y1+h1/2)
                x_c2=int(x2+w2/2)
                y_c2=int(y2+h2/2)
                cv2.rectangle(frame,(x1,y1),(x1+w1,y1+h1),(0,0,255),2)
                cv2.rectangle(frame,(x2,y2),(x2+w2,y2+h2),(0,0,255),2)
                cv2.line(frame, (x_c1,y_c1), (x_c2,y_c2),(0,0,255), 2)
                j=j+1
                
            else:
                #cv2.rectangle(frame,(x1,y1),(x1+w1,y1+h1),(255,0,0),2)
                j=j+1
                
            
    cv2.imshow("Image",frame)
    cv2.imshow("Bird Eye View", bird_image)
    key=cv2.waitKey(1)
    if key == 27:
        break;
        
cap.release()
cv2.destroyAllWindows()

Point detected
[(6, 186)]
Point detected
[(6, 186), (648, 441)]
Point detected
[(6, 186), (648, 441), (349, 5)]
Point detected
[(6, 186), (648, 441), (349, 5), (697, 26)]
Point detected
[(6, 186), (648, 441), (349, 5), (697, 26), (341, 182)]
Point detected
[(6, 186), (648, 441), (349, 5), (697, 26), (341, 182), (339, 259)]
Point detected
[(6, 186), (648, 441), (349, 5), (697, 26), (341, 182), (339, 259), (481, 229)]
